# Ridge regression workflow for Neuroscout

This notebook implements a cross-valided voxel-wise encoding model for a single subject using Regularized Ridge Regression.

The goal is to demonstrate how to obtain Neuroscout data to fit models using custom pipelines. For a comprehensive tutorial, check out the excellent [voxelwise modeling tutorials](https://gallantlab.github.io/voxelwise_tutorials/index.html) from the Gallant Lab.

__Note__: By implementing a custom pipeline, your analysis will not be centrally registered on neuroscout.org, and a reproducible record will not be made. For analyses supported the Neuroscout-CLI (e.g. group voxel-wise mass univariate GLM models), it is recommended to use the neuroscout.org web inteface, or follow the guide for programmatically [creating analyses 
using pyNS](https://pyns.readthedocs.io/en/latest/analyses.html).

### Citing Neuroscout

If you publish any results using the Neuroscout data, be sure to cite the Neuroscout paper, and corresponding datasets:

    Alejandro de la Vega, Roberta Rocca, Ross W Blair, Christopher J Markiewicz, Jeff Mentch, James D Kent, Peer Herholz, Satrajit S Ghosh, Russell A Poldrack, Tal Yarkoni (2022). *Neuroscout, a unified platform for generalizable and reproducible fMRI research*. eLife 11:e79277. https://doi.org/10.7554/eLife.79277
    
    Visconti di Oleggio Castello, M., Chauhan, V., Jiahui, G., & Gobbini, M. I. An fMRI dataset in response to “The Grand Budapest Hotel”, a socially-rich, naturalistic movie. Sci Data 7, 383 (2020). https://doi.org/10.1038/s41597-020-00735-4

## Fetching Neuroscout Data

We can easily retrieve data from _Neuroscout_ using __pyNS__-- the Python Neuroscout API client.
Be sure to refer to the official [pyNS documentation](https://pyns.readthedocs.io/en/latest/) for further usage information, with particular focus on the section on [fetching predictors and images](https://pyns.readthedocs.io/en/latest/fetching.html).

### What data is available?

If you're not sure what is available, you can browse Neuroscout's [datasets](https://neuroscout.org/datasets) and [predictors](https://neuroscout.org/predictors) online.

Here, we were going to focus on the 'Budapest' dataset, choosing the first subject (`sid000005`) as an example. 

In [3]:
# Find first subject for Budapest
from pyns import Neuroscout

api = Neuroscout()

dataset_name = 'Budapest'

# First run for Budapest dataset
first_run = api.runs.get(dataset_name=dataset_name)[5]
subject = first_run['subject'] # Save subject name

first_run

{'acquisition': None,
 'dataset_id': 27,
 'duration': 598.0,
 'id': 1438,
 'number': 1,
 'session': None,
 'subject': 'sid000007',
 'task': 48,
 'task_name': 'movie'}

### Fetching predictors 

We will fetch two sets of predictors: [Mel spetrogram](https://neuroscout.org/predictor/mel_0), and  [Mel Frequency Cepstral Coefficient (MFCC)](https://neuroscout.org/predictor/mfcc_0). Both of these features are extracted from the auditory track of the movie stimulus ('The Grand Budapest Hotel'). Later in the tutorial we'll fit an encoding model to each set of features separately, and then jointly using a banded model.

First, we define the names of the predictors we will fetch. 

To learn more about basic Neuroscout API querying, see this [guide](https://pyns.readthedocs.io/en/latest/querying.html)

In [4]:
mfccs = [f'mfcc_{i}' for i in range(20)]
mel = [f'mel_{i}' for i in range(64)]

confounds = ['rot_x', 'rot_y', 'rot_z', 'trans_x', 'trans_y', 'trans_z',
             'a_comp_cor_00', 'a_comp_cor_01', 'a_comp_cor_02',
             'a_comp_cor_03','a_comp_cor_04','a_comp_cor_05']

all_vars = mfccs + mel

Next, we can use the high-level utility `fetch_predictors` to retrieve these predictors for the target subject, rescaled using unit variance, and resampled to the imaging data's Repetition Time (TR):

This results in a dataframe with predictors, plus meta-data such as file entities (e.g. subjects, runs)

### Fetch fMRI data and loading images

To retrieve Neuroscout data, we use `datalad` to fetch the preprocessed images remote.
pyNS includes a helper function to facilitate installing and fetching the dataset using datalad: `fetch_images`.

Provide the name of the dataset (`Budapest`), plus a directory where your datasets are stored, plus (optionally) filters to restrict which files are downloaded (e.g. subject)

In [ ]:
from pyns.fetch_utils import fetch_images

preproc_dir, img_objs = fetch_images('Budapest', '/tmp/', subject=subject, fetch_brain_mask=True)

/home/alejandro/miniconda3/envs/nsencoding/lib/python3.10/site-packages/bids/layout/validation.py:51: UserWarning: The ability to pass arguments to BIDSLayout that control indexing is likely to be removed in future; possibly as early as PyBIDS 0.14. This includes the `config_filename`, `ignore`, `force_index`, and `index_metadata` arguments. The recommended usage pattern is to initialize a new BIDSLayoutIndexer with these arguments, and pass it to the BIDSLayout via the `indexer` argument.
  warnings.warn("The ability to pass arguments to BIDSLayout that control "
/home/alejandro/miniconda3/envs/nsencoding/lib/python3.10/site-packages/bids/layout/validation.py:156: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


`fetch_images` represents images as pybids `BIDSImageFile` objects, which include meta-data such as entities as part of the object

In [ ]:
img_objs[5].get_entities()

Using the entities, we can separate the list of images into two lists: preprocessed functional images, and brain masks

In [ ]:
all_funcs = [f for f in img_objs if f.entities['suffix'] == 'bold']
all_masks = [f for f in img_objs if f.entities['suffix'] == 'mask']

### Image preprocessing

The following computes a joint mask for all runs. Alternatively, we could also provide a apriori ROI mask here.

Now we can apply the corresponding brain mask to each run using `NiftiMasker` from `nilearn`, and stack them into a single array for subsequent analysis

In [ ]:
import pandas as pd
import numpy as np
from nilearn.maskers import NiftiMasker

def _mask_and_stack_images(image_objects, mask):
    """ Stack images into single array, and collect metadata entities into dataframe """
    masker = NiftiMasker(mask_img=mask)

    arrays = []
    entities = []
    image_objects = sorted(image_objects, key=lambda x: x.entities['run'])
    for img in image_objects:
        run_y = masker.fit_transform(img)
        arrays.append(run_y)
        entities += [dict(img.entities)] * run_y.shape[0]
    entities = pd.DataFrame(entities)
    return np.vstack(arrays), entities, masker

In [ ]:
y, img_entities, masker = _mask_and_stack_images(all_funcs, inter_mask)

The stacked runs have shape: (n_volumes, n_voxels)

In [ ]:
y.shape

We also keep the entities associated with each volume in a dataframe:

In [ ]:
img_entities

## Fitting Ridge regression model using melspectrogram features

First, we'll fit a basic Ridge regression model to a subset of voxels (for demonstration purposes).

We'll define two cross-validators: an outer and an inner cv. The outer cross-validator will loop be used to estimate the performance of the model on unseen data, and the inner cv will be used to select the alpha hyperparameter for Ridge regression, within each fold of the outer cross-validator.

In both cases, we'll use the leave-one-run-out strategy, testing the model on an unseen run. Since there are multiple observations per run, we use the`run` column of `X_entities` to group observations, and ensure they appear together within each fold.

Finally, we also define a scoring function, in this case, `correlation_score`.

Note that we are using the `himalaya` library's `KernelRidgeCV` estimator, as it is optimized for multi-target (i.e. voxel) estimation & hyperparameter optimization with a sklearn-like API.

In [ ]:
from himalaya.ridge import RidgeCV
from sklearn.model_selection import GroupKFold
from himalaya.scoring import correlation_score
from himalaya.backend import set_backend

# backend = set_backend("cpu")

The following is a generic pipeline for applying the esimator to the data (including support for banded-regression, which we'll discuss later)

In [ ]:
def _model_cv(estimator, cv, X_vars, y, bands=None, groups=None,
              scoring=correlation_score,
              inner_cv=None, confounds=None, split=None):
    
    # Container for results
    results = {
        'coefficients': [],
        'test_predictions': [],
        'test_scores': []}

    
    if bands is not None:
        X = []
        for band in bands:
            X.append(X_vars[band].values)
    else:
        X = X_vars.values
        
    n_confounds = confounds.shape[1] if confounds is not None else 0
    
    # If confounds, stack at the end
    if confounds is not None:
        X = X + [confounds] if type(X) == list else np.hstack((X, confounds))
    
    # Extract number of samples for convenience
    n_samples = y.shape[0]
    
    # Loop through outer cross-validation folds
    for train, test in cv.split(np.arange(n_samples), groups=groups):
        
        # Get training model for list of model bands
        X_train = [x[train] for x in X] if type(X) == list else X[train]
        X_test = [x[test] for x in X] if type(X) == list else X[test]
        
        # Create inner cross-validation loop if specified
        if inner_cv:
            # Split inner cross-validation with groups if supplied
            inner_groups = np.array(groups)[train] if groups else groups
            inner_splits = inner_cv.split(np.arange(n_samples)[train],
                                          groups=inner_groups)
            
            # Update estimator with inner cross-validator
            estimator.set_params(cv=inner_splits)
        
        # Fit the regression model on training data
        estimator.fit(X_train, y[train])
        
        # Zero out coefficients for confounds if provided
        if confounds is not None:
            estimator.coef_[-n_confounds:] = 0
        
        # Compute predictions with optional splitting by band
        kwargs = {}
        if split is not None:
            kwargs['split'] = split
            
        test_prediction = estimator.predict(X_test, **kwargs)
        
        # Test scores should also optionally split by band
        test_score = scoring(y[test], test_prediction)
        
        # Populate results dictionary
#         results['coefficients'].append(estimator.dual_coef_)
        results['test_predictions'].append(test_prediction)
        results['test_scores'].append(test_score)
        
    # Combine into single aray
#     results['coefficients'] = np.stack(results['coefficients'])
    results['test_scores'] = np.stack(results['test_scores'])
    
    return results

## Finite impulse response model

Due to the hemodynamic response lag, it's likely the model would perform better if the predictors were delayed.

We can do so using a Finite Impulse Response (fir) model. 

Using `fetch_predictors`, we can ask for the predictors to be returned as a `BIDSVariableCollection`, which enables us to apply any of the transformations implemented in pybids. 

Alternatively, you could apply any arbitrary transformations, by requesting a pandas data-frame. 

In [ ]:
from pyns.fetch_utils import fetch_predictors

collection = fetch_predictors(all_vars + confounds, dataset_name='Budapest', subject='sid000007', 
                              resample=True, return_type='collection')

In [ ]:
import pickle

In [ ]:
pickle.dump(collection, open('ohbm_results/Budapest_sid000005_mel_mfcc_collection.pkl', 'wb'))

In [ ]:
# collection = pickle.load(open('ohbm_results/Budapest_sid000005_mel_mfcc_collection.pkl', 'rb'))

Next, we use `Convolve` to apply a `fir` model, and `Scale` to ensure the final predictors have a mean of zero, and finally convert to a pandas df

In [ ]:
from bids.modeling.transformations import Convolve, Scale

def _fir_scale_df(collection, predictor_names):    
    Convolve(collection, predictor_names, model='fir', fir_delays=[1, 2, 3, 4])
    
    all_vars = list(collection.variables.keys())
    
    Scale(collection, predictor_names, demean=True, rescale=False)

    # To df, and sort rows by keys
    collection_df = collection.to_df().sort_values(['subject', 'run', 'onset'])

    # Reorder columns
    sort_columns = ['onset', 'duration'] + predictor_names
    sort_columns += collection_df.columns.difference(sort_columns).tolist()
    collection_df = collection_df[sort_columns]
    
    return collection_df

collection_df = _fir_scale_df(collection, all_vars)

In [ ]:
# Set up estimator and CV objects
estimator = RidgeCV()

groups = collection_df['run'].tolist()
n_runs = len(set(groups))
cv = GroupKFold(n_splits=n_runs)
inner_cv = GroupKFold(n_splits=n_runs - 1)

The FIR convolution results in 5x amount of predictors, including the delayed version of the predictors.

We can now fit a model with all the mel-spectrogram features, to see if that improves prediction

In [ ]:
X = collection_df.iloc[:, collection_df.columns.str.startswith('mel')]
confounds_X = collection_df[confounds]

In [ ]:
results_mel_fir = _model_cv(estimator, cv, X, y, inner_cv=inner_cv, groups=groups, confounds=confounds_X)

In [ ]:
mean_scores_mel_fir = results_mel_fir['test_scores'].mean(axis=0)
mean_scores_mel_fir[mean_scores_mel_fir < 0] = 0

The max score increases, while the mean score remains similar

In [ ]:
mean_scores_mel_fir.max()

In [ ]:
mean_scores_mel_fir.mean()

In [ ]:
plot_stat_map(masker.inverse_transform(mean_scores_mel_fir), display_mode='z', threshold=0.075, vmax=0.35, cut_coords=[-23, -6, 11, 24, 39])

### FIR model with MFCC features

In [ ]:
X = collection_df.iloc[:, collection_df.columns.str.startswith('mfcc')]
results_mfcc_fir = _model_cv(estimator, cv, X, y, inner_cv=inner_cv, groups=groups, confounds=confounds_X)

In [ ]:
results_mfcc_fir = results_mfcc_fir['test_scores'].mean(axis=0)
results_mfcc_fir[results_mfcc_fir < 0] = 0

In [ ]:
results_mfcc_fir.max()

In [ ]:
results_mfcc_fir.mean()

In [ ]:
plot_stat_map(masker.inverse_transform(results_mfcc_fir), display_mode='z', threshold=0.075, vmax=0.35, cut_coords=[-23, -6, 11, 24, 39])

## Combined model

### Banded model with both mel and MFCC

In [ ]:
# from himalaya.kernel_ridge import MultipleKernelRidgeCV

In [ ]:
# estimator = MultipleKernelRidgeCV()
X = collection_df.iloc[:, collection_df.columns.str.startswith('m')]

results_combined = _model_cv(estimator, cv, X, y, inner_cv=inner_cv, groups=groups, confounds=confounds_X)

In [ ]:
results_combined = results_combined['test_scores'].mean(axis=0)
results_combined[results_combined < 0] = 0

In [ ]:
results_combined.max()

In [ ]:
results_combined.mean()

In [ ]:
plot_stat_map(masker.inverse_transform(results_combined), display_mode='z', threshold=0.075, vmax=0.35, cut_coords=[-23, -6, 11, 24, 39])

### Simple RMS Model

In [ ]:
rms_collection = fetch_predictors(['rms'], dataset_name='Budapest', subject='sid000005', 
                              resample=True, return_type='collection')
rms_collection_df = _fir_scale_df(rms_collection, ['rms'])

In [ ]:
X = rms_collection_df.iloc[:, rms_collection_df.columns.str.startswith('r')]

rms_results = _model_cv(estimator, cv, X, y, inner_cv=inner_cv, groups=groups)

In [ ]:
rms_results_combined = rms_results['test_scores'].mean(axis=0)
rms_results_combined[rms_results_combined < 0] = 0

In [ ]:
plot_stat_map(masker.inverse_transform(rms_results_combined), display_mode='z', threshold=0.05, cut_coords=[-23, -6, 2, 11, 24, 39])

## Speech only

In [ ]:
speech_collection = fetch_predictors(['speech'], dataset_name='Budapest', subject='sid000005', 
                              resample=True, return_type='collection')
speech_collection = _fir_scale_df(speech_collection, ['speech'])

In [ ]:
speech_collection

In [ ]:
X = speech_collection.iloc[:, speech_collection.columns.str.startswith('spee')]

speech_results = _model_cv(estimator, cv, X, y, inner_cv=inner_cv, groups=groups)

In [ ]:
speech_results = speech_results['test_scores'].mean(axis=0)
speech_results[speech_results < 0] = 0

In [7]:
plot_stat_map(masker.inverse_transform(speech_results), display_mode='z', threshold=0.05, cut_coords=[-23, -6, 2, 11, 24, 39])

NameError: name 'plot_stat_map' is not defined

### TODO
- Add motion regressor

In [6]:
print('hi')

hi
